<a href="https://colab.research.google.com/github/cool250/nlp/blob/main/PyTorch_hello_world.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn

import numpy as np

from sklearn.datasets import make_blobs
from matplotlib import pyplot

In [9]:
samples = 5000
train_split = int(samples*0.8)

X,y = make_blobs(n_samples=samples, centers=2, n_features=64, cluster_std=10, random_state=2020)
y = y.reshape(-1,1)

X,y = torch.from_numpy(X), torch.from_numpy(y)
X,y = X.float(), y.float()

X_train, X_test = X[:train_split], X[train_split:]
y_train, y_test = y[:train_split], y[train_split:]

#Print shapes of each dataset
print("X_train.shape:",X_train.shape)
print("X_test.shape:",X_test.shape)
print("y_train.shape:",y_train.shape)
print("y_test.shape:",y_test.shape)
print("X.dtype",X.dtype)
print("y.dtype",y.dtype)

X_train.shape: torch.Size([4000, 64])
x_test.shape: torch.Size([1000, 64])
Y_train.shape: torch.Size([4000, 1])
y_test.shape: torch.Size([1000, 1])
X.dtype torch.float32
y.dtype torch.float32


In [16]:
class NeuralNetwork(nn.Module):

  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(64, 256)
    self.relu1 = nn.ReLU()
    self.fc2 = nn.Linear(256,1024)
    self.relu2 = nn.ReLU()
    self.fc3 = nn.Linear(1024,1)
    self.final = nn.Sigmoid()

  def forward(self,x):
    op = self.fc1(x)
    op = self.relu1(op)
    op = self.fc2(op)
    op = self.relu2(op)
    op = self.fc3(op)
    y = self.final(op)
    return y

In [26]:
def train_network(model, optimizer, loss_function, num_epochs, batch_size, X_train, y_train):
  model.train()
  loss_across_epochs = []
  for epoch in range(num_epochs):
    train_loss = 0.0

    for i in range(0, X_train.shape[0], batch_size):
      input_data = X_train[i:min(X_train.shape[0], i+batch_size)]
      labels = y_train[i:min(X_train.shape[0], i+batch_size)]

      optimizer.zero_grad()

      output = model(input_data)

      loss = loss_function(output, labels)

      loss.backward()

      optimizer.step()
      train_loss += loss.item() * batch_size

    print("Epoch: {} - Loss {:4f}".format(epoch+1, train_loss))
    loss_across_epochs.extend([train_loss])


In [27]:
# Create an object of Neural Network class
model = NeuralNetwork()

#Loss Function
loss_function = nn.BCELoss() # Binary Cross Entropy

# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

# Define epochs and batch size
num_epochs = 10
batch_size = 16

loss = train_network(model, optimizer, loss_function, num_epochs, batch_size, X_train, y_train)

print("Loss - {}".format(loss))

Epoch: 1 - Loss 129.714970
Epoch: 2 - Loss 23.325585
Epoch: 3 - Loss 1.234440
Epoch: 4 - Loss 9.276546
Epoch: 5 - Loss 0.335508
Epoch: 6 - Loss 0.097684
Epoch: 7 - Loss 0.001499
Epoch: 8 - Loss 0.001414
Epoch: 9 - Loss 0.001330
Epoch: 10 - Loss 0.001248
Loss - None
